In [1]:
from bokeh.io import output_notebook
import os ,sys, glob
import numpy as np, pandas as pd, scipy as sp
from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold

from cvopt.model_selection import SimpleoptCV
from cvopt.search_setting import search_category, search_numeric

output_notebook() 

Loading BokehJS ...

In [2]:
import lightgbm as lgb
sys.path.append('../../../github/module')
from load_data import load_data, x_y_split
from preprocessing import set_validation, split_dataset, get_categorical_features, get_numeric_features, dframe_dtype, factorize_categoricals, make_feature_set
from load_data import pararell_load_data, x_y_split

In [3]:
unique_id = 'SK_ID_CURR'
target = 'TARGET'
base = pd.read_csv('../data/base_train_test.csv')

' 学習に使うfeature_setをmerge '
path = '../features/3_winner/*.npy'
dataset = make_feature_set(base, path)
# dataset.set_index(unique_id, inplace=True)

categorical = get_categorical_features(dataset, [])
dataset = factorize_categoricals(dataset, categorical)

train = dataset.query('is_train==1')
test = dataset.query('is_test==0')

train.drop(['is_train', 'is_test'], axis=1, inplace=True)
test.drop(['is_train', 'is_test'], axis=1, inplace=True)

train, valid = split_dataset(train, 1)
x_train, y_train = x_y_split(train, target)
x_val, y_val = x_y_split(valid, target)

In [7]:
for col in x_train.columns:
    print(col)

SK_ID_CURR
a_AMT_ANNUITY
a_AMT_ANNUITY_mean@['CODE_GENDER',_'NAME_EDUCATION_TYPE']
a_AMT_ANNUITY_mean@['CODE_GENDER',_'NAME_FAMILY_STATUS']
a_AMT_ANNUITY_mean@['CODE_GENDER',_'NAME_INCOME_TYPE']
a_AMT_ANNUITY_mean@['NAME_EDUCATION_TYPE',_'NAME_INCOME_TYPE']
a_AMT_ANNUITY_mean@['NAME_FAMILY_STATUS',_'OCCUPATION_TYPE']
a_AMT_ANNUITY_mean@['NAME_INCOME_TYPE',_'OCCUPATION_TYPE']
a_AMT_ANNUITY_mean@['NAME_INCOME_TYPE',_'ORGANIZATION_TYPE']
a_AMT_ANNUITY_std@['CODE_GENDER',_'NAME_EDUCATION_TYPE']
a_AMT_CREDIT
a_AMT_CREDIT_mean@['NAME_FAMILY_STATUS',_'OCCUPATION_TYPE']
a_AMT_CREDIT_std@['CODE_GENDER',_'NAME_INCOME_TYPE']
a_AMT_CREDIT_sum@['NAME_EDUCATION_TYPE',_'OCCUPATION_TYPE']
a_AMT_GOODS_PRICE
a_AMT_INCOME_TOTAL
a_AMT_REQ_CREDIT_BUREAU_MON
a_AMT_REQ_CREDIT_BUREAU_QRT
a_AMT_REQ_CREDIT_BUREAU_YEAR
a_APARTMENTS_AVG
a_BASEMENTAREA_AVG
a_CNT_FAM_MEMBERS
a_COMMONAREA_AVG
a_DAYS_BIRTH
a_DAYS_BIRTH_mean@['NAME_EDUCATION_TYPE',_'NAME_FAMILY_STATUS']
a_DAYS_BIRTH_mean@['NAME_EDUCATION_TYPE',_'OCCUP

In [4]:
# サーチ範囲min, max, 変数のタイプ("int" or "float")
param_distributions = {
    "C": search_numeric(0.01, 3.0, "float"), 
    "tol" : search_numeric(0.0001, 0.001, "float"),  
    "penalty": search_category(['l1', 'l2']),  # サーチ対象カテゴリのリスト
    "class_weight" : search_category([None, "balanced"]),
    }

estimator = lgb.LGBMClassifier()
# estimator = lgb.Dataset(data=x_train, label=y_train)
# lgb_eval = lgb.Dataset(data=x_val, label=y_val)

param_distributions = {
    'min_child_weight': search_numeric(7, 17 , "float"),
    'subsample': search_numeric(0.5, 1.0, "float"),
    'bagging_freq': search_numeric(0, 1, "int"),
    'n_estimators': search_numeric(100, 10000, "int"),
    'colsample_bytree': search_numeric(0.5, 1.0, "float"),
#     'learning_rate': 0.1,
#     'max_depth': -1,
    'min_data_in_bin': search_numeric(5, 15, "int"),
    'min_split_gain': search_numeric(0, 1, "float"),
    'clf_alpha': search_numeric(0, 1.0, "float")
    ,
    'max_bin': search_numeric(100, 500, "int"),
    'num_leaves': search_numeric(4, 63, "int")
#     ,
#     'objective': 'binary'
}


In [5]:

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

opt = SimpleoptCV(estimator, param_distributions, 
                 scoring="roc_auc", 
                 cv=cv, 
                 max_iter=32, 
                 n_jobs=1, 
                 backend="hyperopt", 
                 logdir="./output/", # ログの保存ディレクトリ。これが指定されるとログを保存する。
                 model_id="search_usage", # ログ管理で使用するid。
                 save_estimator=2,        # estimatorの保存設定
                                          #    0: 保存しない
                                          #    1: cvの各foldでfitしたestimatorを保存
                                          #    2: 1に加え、train_data全体でfitしたestimatorを保存
                 verbose=2,               # サーチログの表示設定。
                                          #    0: 非表示
                                          #    1: 最高スコアとサーチ完了時間（推定）をテキストで表示。
                                          #    2: サーチログを可視化して表示
                 )



In [6]:
opt.fit(x_train, y_train, validation_data=(x_val, y_val))   # validation_data

HyperoptCV(algo=<function suggest at 0x7fa05f71d620>, cloner='sklearn',
      cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
      estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1),
      logdir='./output/', max_iter=32, model_id='search_usage', n_jobs=1,
      param_distributions={'min_child_weight': {'valtype': 'numeric', 'low': 7, 'high': 17, 'dtype': 'float'}, 'subsample': {'valtype': 'numeric', 'low': 0.5, 'high': 1.0, 'dtype': 'float'}, 'bagging_freq': {'valtype': 'numeric', 'low': 0, 'high': 1, 'dtype': 'int'}, 'n_estimators': {'valtype': 'numeric',...': 500, 'dtype': 'int'}, 'num_leaves': {'valtype': 'numeric',

In [ ]:
pd.DataFrame(opt.cv_results_)